# 创造训练数据

In [158]:
# 聚合所有数据
import glob
import pandas as pd

X = []
y = []
for in_file in glob.glob("data/tokens.train/*.txt"):
    print(in_file)
    for line in open(in_file):
        label, sentense = line.strip().split("\t")
        if label == "0"2
            label = "FF"
        elif label == "1":
            label = "MP"
        else:
            assert "Error"
        X.append(sentense)
        y.append(label)

data = {}
data["label"] = y
data["text"] = X
pd.DataFrame(data).to_csv("data/train.csv", index=None)

data/tokens.train/tokens.08.txt
data/tokens.train/tokens.05.txt
data/tokens.train/tokens.04.txt
data/tokens.train/tokens.06.txt
data/tokens.train/tokens.07.txt


In [159]:
import torchtext

TEXT = torchtext.data.Field()
LABEL = torchtext.data.LabelField()

my_data = torchtext.data.TabularDataset(path='data/train.csv',
                                        format='csv', 
                                        fields=[('labels', LABEL), ('text', TEXT)])

# train_data, valid_data = my_data.split(split_ratio=0.8)
# train_data, valid_data

In [160]:
TEXT.build_vocab(my_data, min_freq=10)
LABEL.build_vocab(my_data, min_freq=10)

In [161]:
len(TEXT.vocab), len(LABEL.vocab)

(21142, 2)

In [162]:
LABEL.vocab.stoi

defaultdict(None, {'MP': 0, 'FF': 1})

In [163]:
train_data, valid_data = my_data.split(split_ratio=0.8)
# train_data, valid_data

train_iter, valid_iter = torchtext.data.iterator.BucketIterator.splits((train_data, valid_data),
                                             batch_sizes=(32, 32),
                                             sort_key=lambda x: len(x.text), # field sorted by len
                                             sort_within_batch=True,
                                             repeat=False)

In [164]:
batch = next(iter(train_iter))
print(train_iter)
print('batch:\n', batch)
print('batch_text:\n', batch.text)
print('batch_label:\n', batch.label)

batch:
 
[torchtext.data.batch.Batch of size 32]
	[.labels]:[torch.LongTensor of size 32]
	[.text]:[torch.LongTensor of size 15x32]
batch_text:
 tensor([[   19,  5547, 13388,    33,     2, 19448,   327, 15408,  3864,  6563,
          3342,     2,  7627,     0,    39, 15388,   385,  3303,     2,  3085,
          7258,     0, 20170,   840,  3712,     0,    28,     2, 12502,  9566,
          1610, 19891],
        [   40,     2,    13,    68,     5,  8020,   242, 11547,   751,     3,
             2,    11,    95,    10,   433,     2,   932,   118,     8,   599,
            97,  3921,   551,    24,    28,     6,   258,     2,    13,    95,
           338,     2],
        [    4,     2, 10746,     3,    25,    31,   230,   293,   751,   226,
             2,   208,   583,  4396,    11,    11,    24,   179,     0,    11,
             3,  2735,    77,  1693,   950,   477,   492,  4082,   123, 15493,
            28,     2],
        [   19,     2,  7299,   519,   185,    67,   454,   512,   751, 

AttributeError: 'Batch' object has no attribute 'label'

In [ ]:
# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)


# training_data = [
#     ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
# ]
# word_to_ix = {}
# for sent, tags in training_data:
#     for word in sent:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)
# print(word_to_ix)
# tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# # These will usually be more like 32 or 64 dimensional.
# # We will keep them small, so we can see how the weights change as we train.
# EMBEDDING_DIM = 6
# HIDDEN_DIM = 6

# 模型

In [166]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(23)


class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [168]:
VOCAB_SIZE = len(TEXT.vocab)
EMBED_DIM = 32
NUN_CLASS = 2
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)

# 训练

In [ ]:
def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    # data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
    #                   collate_fn=generate_batch)
    # for i, (text, offsets, cls) in enumerate(data):
    #     optimizer.zero_grad()
    #     text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
    #     output = model(text, offsets)
    #     loss = criterion(output, cls)
    #     train_loss += loss.item()
    #     loss.backward()
    #     optimizer.step()
    #     train_acc += (output.argmax(1) == cls).sum().item()
    for batch in train_iter:
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(batch)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()    

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

# 验证